### Tips
- 仓位的单位是金额而不是份数
- 展期收益可以从两个角度来理解
    - 当你持有一个期货合约（而非现货）时，所获得的超额收益，就叫做展期收益。
    - 展期收益反映了换月时期货合约切换带来的额外收益或损失。

    无论怎么理解本质上都是当下的期货价格-现货价格，即当下的升贴水大小，在第两种语境下期货价格收敛于现货价格确保了这一结论的成立。
- IH 与 IF 为大盘蓝筹风格；IC 与 IM 为中盘成长风格


### 困惑
1. 理论上 P3 的表格中应该满足期货收益=指数收益+展期收益，但是实际却接近但不符合这一规律，如何理解。包括累计收益的计算。后续有做复现，不知道理解是否正确。
2. 市值因子是什么？如何计算的？为什么说会等资金配置会有很大的市值因子暴露。
3. P7为什么说品种间相关性较小时会导致 beta 风险对冲效果不佳。

# 1. 前提假设研究

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['FZKai-Z03S']
mpl.rcParams['axes.unicode_minus'] = False
from config import *
from data_utils import *

| 代码     | 名称         | 对应指数         | 风格         | 对应ETF代码              |指数代码|
| ------ | ---------- | ------------ | ---------- | -------------------- | ----- |
| **IF** | 沪深300股指期货  | **沪深300指数**  | 大盘、主流蓝筹    | 510300.SH（华泰柏瑞沪深300） |000300.SH |
| **IH** | 上证50股指期货   | **上证50指数**   | 超大盘、金融地产   | 510050.SH（华夏上证50）    |000016.SH |
| **IC** | 中证500股指期货  | **中证500指数**  | 中盘成长、广义中小盘 | 510500.SH（南方中证500）   |000905.SH |
| **IM** | 中证1000股指期货 | **中证1000指数** | 小盘成长、弹性大   | 512100.SH（中证1000ETF） |000852.SH |


- 中证指数作为指数发布方，提供指数；
- 各 ETF 发行方（如华夏、华泰柏瑞、南方）复制跟踪该指数；
- 在不同交易所（沪深）挂钩的是“ETF”，不是指数本身。

✅ 指数值在不同市场是相同的；

✅ ETF价格可能不同，因其受交易供需、流动性、折溢价等影响。

1. 不同股指间本身有一定的相关性：根据折线图直观感受和相关系数矩阵结果可以表明确实四个产品之间具有比较高的相关性，但是根据投资组合理论对于相关性高的产品其实波动率降低能力有限，后续需观察这一现象是否在策略上有体现。
2. IH/IF 与 IC/IM 是相关性较高的两个配对：因为 IH 与 IF 都是大盘蓝筹风格，IC 与 IM 都是中小盘成长风格。

In [ ]:
#该部分可以用 pivot_index_data 替换
index_df=pd.read_csv(INDEX_DATA)
HS300_df=index_df[index_df['thscode']=='000300.SH']
ZZ500_df=index_df[index_df['thscode']=='000905.SH']
ZZ1000_df=index_df[index_df['thscode']=='000852.SH']
SZ50_df=index_df[index_df['thscode']=='000016.SH']
index_df = pd.DataFrame({
    '000300.SH': HS300_df['close'].reset_index(drop=True),
    '000905.SH': ZZ500_df['close'].reset_index(drop=True),
    '000852.SH': ZZ1000_df['close'].reset_index(drop=True),
    '000016.SH': SZ50_df['close'].reset_index(drop=True)
})

In [ ]:
# 重置index并创建新的DataFrame
corr_matrix = index_df.corr()
print(corr_matrix)

3. 不同品种间的升贴水有很大差异，基差可能贡献大部分跨品种组合的收益（这一观点由收益分解结果支持）：本文所提到的这种升贴水差异主要体现在 futures_data 文件的 ICIF、IFIH、IMIC、IMIH 的数值上，这表示的是年化基差率的差异，权重是基于本金计算的。
- 理论基础：跨品种套利组合的总收益分解【这一现实情况决定了在进行策略信号构建时需要考虑分开预测大小盘风格于展期收益】
    - 指数相对涨跌的收益（style effect，风格轮动）：不同风格（如大盘 vs 小盘、价值 vs 成长）在不同时期表现交替领先或落后，从而形成了可以利用的轮动机会。
    - 展期收益（基差收敛带来的收益）：当你持有一个期货合约（而非现货）时，所获得的超额收益，就叫做展期收益，具体来说比如你持有的是期货合约而不是现货，额外多赚了 50 点，这部分就叫展期收益。展期收益 = 期货价格变动 – 现货价格变动，$展期收益=（F_t^{平仓}-F_0^{建仓}）-（S_t-S_0）$，由于平仓期货价格收敛于现货价格，所以展期收益本质上就是基差，即升贴水的值。

【份数相同情景】组合总收益=
$(F_A^{t_1}-F_A^{t_0})-(F_B^{t_1}-F_B^{t_0})=$

$(S_A^{t_1}+基差_A^{t_1}-S_A^{t_0}-基差_A^{t_0})-(S_B^{t_1}+基差_B^{t_1}-S_B^{t_0}-基差_B^{t_0})=$

$(S_A^{t_1}-S_A^{t_0})-(S_B^{t_1}-S_B^{t_0})+(基差_A^{t_1}-基差_A^{t_0})-(基差_B^{t_1}-基差_B^{t_0})=$

$\Delta S_A-\Delta S_B+(基差_A^{t_1}-基差_B^{t_1})-(基差_A^{t_0}-基差_B^{t_0})$

- 结论：
1. 在实操的时候构建信号将「风格轮动预测」和「基差与指数信号」分开考虑，而后合成为跨品种套利的策略信号。
    1. 风格轮动对应的是$\Delta S_A-\Delta S_B$部分
    2. 如果持有至接近期限，那么$(基差_A^{t_1}-基差_B^{t_1})$会很接近于 0，此时问题的关键在于建仓时的基差的差额
2. 因为 IH/IF 与 IC/IM 是两个相关性较高的配对，所以策略也可以被分为同风格的跨品种套利和跨风格的跨品种套利。
3. 当金额相同时基差的差转为了基差率的差

In [ ]:
futures_data=pd.read_csv(FUTURES_DATA)
futures_df = pd.DataFrame({
    'ICIF': futures_data['ICIF'],
    'IFIH': futures_data['IFIH'],
    'IMIC': futures_data['IMIC'],
    'IMIH': futures_data['IMIH']
})
futures_df.index = futures_data['time']

4. 四个品种的跨品种套利可以有两种基本思路：时间序列和横截面
    1. 时间序列的含义是确定组合方式（视作一个资产比如 ICIF ），策略核心在于通过因子择时买入和卖出
    2. 横截面思路是组合方式不确定，但是策略持有期固定（未必，待补充），策略核心在于通过因子确定各资产买入权重。

5. 主力合约的收益分解

In [ ]:
index_df=pd.read_csv(INDEX_DATA)
index_pivot_df=pivot_index_data(index_df,CODE_MAP)
futures_df=pd.read_csv(FUTURES_DATA)
futures_df['time'] = pd.to_datetime(futures_df['time'])
futures_df['IC_LTDATE_NEW']=pd.to_datetime(futures_df['IC_LTDATE_NEW'])
futures_df['IF_LTDATE_NEW']=pd.to_datetime(futures_df['IF_LTDATE_NEW'])
futures_df['IH_LTDATE_NEW']=pd.to_datetime(futures_df['IH_LTDATE_NEW'])
futures_df['IM_LTDATE_NEW']=pd.to_datetime(futures_df['IM_LTDATE_NEW'])
total_df= pd.merge(futures_df, index_pivot_df, on='time', how='left')
IFIH_df = pd.DataFrame({
    'time': total_df['time'],
    'index_return': total_df['IF_udy_changeRatio']-total_df['IH_udy_changeRatio'],
    'futures_return': total_df['IF_PCT_CHG']-total_df['IH_PCT_CHG'],
    'basis_return': total_df['IFIH']/244
})
IFIH_df['index_nv']=(1+IFIH_df['index_return']).cumprod()
IFIH_df['futures_nv']=(1+IFIH_df['futures_return']).cumprod()
IFIH_df['basis_nv']=(1+IFIH_df['basis_return']).cumprod()

# 2. 根据基差构建夸品种套利信号
## 2.1 时序思路
- 思路：等资金做多一个品种并做空一个品种，根据主力合约年化基差率之差构建跨品种套利仓位信号（具体来说计算差值后取相反数*10）
- 起源：跨品种期货收益分解公式，根据公式可知在其他条件不变的情况下，基差率之差越大，收益越大，所以基差率差的大小可以作为一种仓位信号。

## 2.2 基于波动率中性改进的时序思路
- 思路：等资金的一个隐患是有较大的市值风险因子暴露

In [ ]:
index_data=pd.read_csv(INDEX_DATA)
df_futures = pd.read_csv(FUTURES_DATA)
df_futures['time'] = pd.to_datetime(df_futures['time'])
df_futures = df_futures.set_index('time')

In [ ]:
df_index_return = pd.pivot(index_data, index="time", columns="thscode", values="changeRatio") / 100
df_index_return.index = pd.to_datetime(df_index_return.index)

In [ ]:
columns = df_index_return.columns
for col in columns:
    # 计算每个标的的滚动60日波动率
    df_index_return[f'{col}_60d_volatility'] = df_index_return[col].rolling(window=60).std()

In [ ]:
df_vol = df_index_return.copy(deep=True)
df_vol.rename(columns=lambda x: x.replace('000016.SH', 'IH').replace('000905.SH', 'IC').replace('000300.SH', 'IF').replace('000852.SH', 'IM'), inplace=True)
pairs = ["IFIH", "ICIF", "IMIC", "IMIH"]

In [ ]:
for pair in pairs:
    # 计算多头标的权重
    df_vol["{}_{}_weight".format(pair, pair[:2])] = 2 * df_vol["{}_60d_volatility".format(pair[2:])] / (
            df_vol["{}_60d_volatility".format(pair[:2])] + df_vol["{}_60d_volatility".format(pair[2:])])
    # 计算空头标的权重
    df_vol["{}_{}_weight".format(pair, pair[2:])] = 2 * df_vol["{}_60d_volatility".format(pair[:2])] / (
            df_vol["{}_60d_volatility".format(pair[:2])] + df_vol["{}_60d_volatility".format(pair[2:])])
    # 计算收益率60日相关性
    df_vol["{}_60d_corr".format(pair)] = df_vol["{}".format(pair[:2])].rolling(window=60).corr(df_vol["{}".format(pair[2:])])

In [ ]:
df_basis_signal = pd.merge(df_vol, df_futures, left_index=True, right_index=True)
for pair in pairs:
    # 计算基差信号
    df_basis_signal["signal_{}".format(pair)] = (df_basis_signal["{}_{}_weight".format(pair, pair[:2])] * \
                                                df_basis_signal["{}_ANAL_BASISANNUALYIELD".format(pair[:2])] - df_basis_signal["{}_{}_weight".format(pair, pair[2:])] * \
                                                df_basis_signal["{}_ANAL_BASISANNUALYIELD".format(pair[2:])]) / 100
    # 如果收益率60日相关性小于0.7，则空仓
    df_basis_signal["signal_{}_position".format(pair)] = np.where(
        df_basis_signal["{}_60d_corr".format(pair)] >= 0.7,
        -df_basis_signal["signal_{}".format(pair)] * 10,
        0)

In [ ]:
df_basis_signal["date"] = pd.to_datetime(df_basis_signal.index)
df_basis_signal = df_basis_signal[['date', 'signal_IFIH', 'signal_IFIH_position',
                                'signal_ICIF', 'signal_ICIF_position', 'signal_IMIC',
                                'signal_IMIC_position', 'signal_IMIH', 'signal_IMIH_position']]

df_basis_signal.to_csv("basis_signal.csv", index=False)

In [ ]:
import pandas as pd
import os
from config import *
def calculate_nv_data(df_underlying: pd.DataFrame,
                    cal_type: str,
                    code_map: dict = CODE_MAP,
                    available_pairs: list = AVAILABLE_PAIRS) -> pd.DataFrame:
    """
    输入：
        df_underlying: 包含 'time' 列 和对应后缀的原始 Returns 数据
        type: 'index' 或 'futures'
    返回：
        df_nv: index，由各配对组合的净值序列组成，列名形如 'IFIH_index_nv' 或 'IFIH_futures_nv'
    """
    # 根据 type 选列后缀 & 文件名前缀
    if cal_type == 'index':
        return_cols = {v: f"{v}_udy_changeRatio" for v in code_map.values()}
        file_prefix = 'index_nv_data'
    elif cal_type == 'futures':
        return_cols = {v: f"{v}_PCT_CHG" for v in code_map.values()}
        file_prefix = 'futures_nv_data'
    else:
        raise ValueError("type 必须是 'index' 或 'futures'")

    # 时间列转 datetime 并设为索引
    df = df_underlying.copy()
    df["time"] = pd.to_datetime(df["time"])
    df.set_index("time", inplace=True)

    # 存放所有组合净值
    df_nv = pd.DataFrame(index=df.index)

    for pair in available_pairs:
        first, second = pair[:2], pair[2:]
        col_f, col_s = return_cols[first], return_cols[second]
        # 累积净值
        cum_nv = (1 + (df[col_f] - df[col_s])).cumprod()
        # 归一化到 1
        df_nv[f"{pair}_{cal_type}_nv"] = cum_nv / cum_nv.iloc[0]

    # 自动用时间范围拼文件名，并在初次运行时保存
    start_str = df_nv.index[0].strftime('%Y%m%d')
    end_str   = df_nv.index[-1].strftime('%Y%m%d')
    out_path = os.path.join(RAW_DATA_PATH,
                            f"{file_prefix}_{start_str}_{end_str}.csv")
    df_nv.to_csv(out_path, index=True)

    return df_nv

index_data = pd.read_csv(INDEX_DATA)
index_nv=calculate_nv_data(index_data, 'index')

In [ ]:
from data_utils import *
def get_nv_data(df_underlying: pd.DataFrame, cal_type: str) -> pd.DataFrame:
    if search_file_recursive(RAW_DATA_PATH, f"{cal_type}_nv_data.csv"):
        return pd.read_csv(os.path.join(RAW_DATA_PATH, f"{cal_type}_nv_data.csv"))
    else:
        df_nv = calculate_nv_data(df_underlying, cal_type)
        return df_nv

In [ ]:
a=get_nv_data(index_data, 'index')

In [ ]:
def generate_ohlc(df_nv:pd.DataFrame,prefix:str)->pd.DataFrame:
    """
    从净值数据中提取以指定前缀开头的列，并生成 OHLC 表格。

    参数：
    df_nv (pd.DataFrame)：以时间索引的净值表，包含如 'IFIH_index_nv'、'IFIH_futures_nv' 等
    prefix (str)        ：要模糊搜索的列前缀，例如 'IFIH'

    返回：
    pd.DataFrame：包含 ['date','close','open','high','low'] 的新表格
    """
    # 在列名中进行模糊搜索
    matched_cols = [col for col in df_nv.columns if col.startswith(prefix)]
    if not matched_cols:
        raise ValueError(f"没有找到以 '{prefix}' 开头的列")
    # 如果有多个匹配，默认使用第一个
    target_col = matched_cols[0]
    # 生成 OHLC 表格
    df = df_nv[[target_col]].copy().reset_index()
    df.rename(columns={target_col: 'close'}, inplace=True)
    df['open']=df['close']-0.01
    df['high']=df['close']+0.01
    df['low']=df['close']-0.02
    return df

In [ ]:
def calculate_basis_signal(
        df_index:pd.DataFrame,
        df_future:pd.DataFrame,
        pair: str,
        window: int=60,
        corr_threshold:float=0.7,
        leverage:float=10
        )->pd.DataFrame:
    """
    对单个多空组合（pair，如'IFIH'），计算基差信号及仓位。

    参数:
    df_index: pd.DataFrame，包含指数数据，包括时间和指数收盘价以及指数收益。
    df_future: pd.DataFrame，包含期货数据，包括时间和期货收盘价等。
    pair: str，多空组合的代码，如'IFIH'。
    window: int，计算滚动波动率的窗口大小和相关性的窗口大小，默认为60。
    corr_threshold: float，当收益率相关性低于此阈值时强制平仓，默认为0.7。
    leverage: float，信号乘数，默认为10。

    返回:
    pd.DataFrame，包含时间、基差信号和仓位。
        包含三列：
        - date                  (datetime)
        - signal_{pair}         (float) 基差信号
        - signal_{pair}_position(float) 最终仓位信号
    """
    # 计算滚动波动率
    sym_long, sym_short = pair[:2], pair[2:]
    vol_long = df_index[f"{sym_long}_udy_changeRatio"].rolling(window).std()
    vol_short = df_index[f"{sym_short}_udy_changeRatio"].rolling(window).std()

    # 计算当日权重
    weight_long = 2*vol_short / (vol_long + vol_short)
    weight_short = 2*vol_long / (vol_long + vol_short)

    # 计算收益率相关性
    corr = df_index[f"{sym_long}_udy_changeRatio"].rolling(window).corr(df_index[f"{sym_short}_udy_changeRatio"])

    # 计算基差信号
    basis_long = df_future[f"{sym_long}_ANAL_BASISANNUALYIELD"]/100
    basis_short = df_future[f"{sym_short}_ANAL_BASISANNUALYIELD"]/100
    raw_signal = weight_long * basis_long - weight_short * basis_short

    signal = raw_signal.reindex(df_future.index)
    corr_aligned = corr.reindex(df_future.index)
    # 仓位计算
    position = (signal * (-leverage)).where(corr_aligned >= corr_threshold, 0)

    # 组装输出
    df_out = pd.DataFrame({
        'date':                    df_future.index,
        f"signal_{pair}":          signal.values,
        f"signal_{pair}_position": position.values
    }).reset_index(drop=True)

    # 返回结果
    return df_out
def calculate_all_basis_signal(index_data, futures_data, available_pairs:list=AVAILABLE_PAIRS):
    all_signals=[]
    for pair in available_pairs:
        df_sig = calculate_basis_signal(index_data, futures_data, pair)
        all_signals.append(df_sig.set_index('date'))
    df_all_sig = pd.concat(all_signals, axis=1).reset_index()
    return df_all_sig


In [ ]:
index_data = pd.read_csv(INDEX_DATA)
index_data['time'] = pd.to_datetime(index_data['time'])
index_data.set_index('time', inplace=True)

In [ ]:
future_data = pd.read_csv(FUTURES_DATA)
future_data['time'] = pd.to_datetime(future_data['time'])
future_data.set_index('time', inplace=True)

In [ ]:
c=calculate_all_basis_signal(index_data, future_data)

In [1]:
from signal_utils import *
from data_utils import *
from config import *
from datetime import datetime
index_data = pd.read_csv(INDEX_DATA)
index_nv_data=get_nv_data(index_data,"index")

In [4]:
def aggregate_to_monthly_price_change(df):
    """
    为每个月计算多空组合的净值涨跌情况
    :param df: 多空组合后的净值
    :return: 月度涨跌数据
    """
    df = df.copy()
    df["time"] = pd.to_datetime(df["time"])
    df['year'] = df['time'].dt.year
    df['month'] = df['time'].dt.month
    # 用groupby+agg优化
    monthly = df.groupby(['year', 'month']).agg(
        first_price=('close', 'first'),
        last_price=('close', 'last'),
        last_date=('time', 'last')
    ).reset_index()
    monthly['return'] = (monthly['last_price'] / monthly['first_price']) - 1
    monthly['price_up'] = monthly['return'] > 0
    return monthly[['year', 'month', 'return', 'price_up', 'last_date']]

# 函数：计算到某年某月为止的历史价格涨跌胜率

def calculate_price_up_win_rate(df, current_year, current_month):
    # 只包含当前月之前的历史数据
    mask = ((df['year'] < current_year) | ((df['year'] == current_year) & (df['month'] < current_month)))
    historical_data = df[mask].copy()
    # 按月份分组计算价格上涨的胜率
    monthly_win_rates = {}
    for month in range(1, 13):
        month_data = historical_data[historical_data['month'] == month]
        if len(month_data) > 0:
            wins = month_data['price_up'].sum()
            total = len(month_data)
            win_rate = wins / total if total > 0 else 0.5
            monthly_win_rates[month] = win_rate
        else:
            monthly_win_rates[month] = 0.5  # 如果没有历史数据，默认为50%
    return monthly_win_rates

# 计算季节性信号和权重

def calculate_seasonal_signals(monthly_data, start_year, end_year):
    results = []
    from datetime import datetime
    now = datetime.now()
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            # 跳过未来数据（只到本月）
            if (year > now.year) or (year == now.year and month > now.month):
                continue
            # 计算该月的历史价格上涨胜率
            win_rates = calculate_price_up_win_rate(monthly_data, year, month)
            current_win_rate = win_rates[month]
            deviation = current_win_rate - 0.5
            signal_direction = 1 if deviation > 0 else -1  # 1表示多头，-1表示空头
            weight = abs(deviation)
            results.append({
                'year': year,
                'month': month,
                'win_rate': current_win_rate,
                'deviation': deviation,
                'signal': signal_direction,
                'weight': weight
            })
    return pd.DataFrame(results)

In [6]:
import pandas as pd
from config import *
from data_utils import *

In [8]:
index_data = pd.read_csv(INDEX_DATA)
index_nv_data = get_nv_data(index_data, "index")
for pair in AVAILABLE_PAIRS:
    daily_data = generate_ohlc(index_nv_data, pair)
    monthly_data = aggregate_to_monthly_price_change(daily_data)
    # 生成信号
    start_year = 2010  # 从2010年开始生成信号
    end_year = 2024  # 到2023年结束
    signals = calculate_seasonal_signals(monthly_data, start_year, end_year)
    signals[["month", "win_rate", "signal", "weight"]].to_csv("../data/season_signal_test_{}.csv".format(pair), index=False)

- 所以说整个代码逻辑其实很简单，中间过程是怎么样的无所谓，最终就是要搞明白生成的交易信号是什么？交易信号对应的交易逻辑是什么就可以了！b